# Rank BM 25 Search Engine

## Configuration

In [1]:
%load_ext autotime

In [2]:
from rank_bm25 import *

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import os

In [3]:
path = os.getcwd()+'\\full_data\\'
print('Path: ', path)

Path:  C:\Users\ramos\OneDrive\Documents\SyracuseUniversity\9th_Quarter\IST736\Project\full_data\


## Analysis

### About the Data

In [4]:
df = pd.read_csv(path + 'mcu_data.csv', index_col=0).reset_index(drop=True)[['character', 'line', 'movie', 'year', 'words']]
print('Entries: ', len(df))
df.head()

Entries:  6509


,character,line,movie,year,words
0,TONY STARK,"Oh, I get it. You guys aren’t allowed to talk...",Iron Man,2008,22
1,TONY STARK,Oh. I see. So it’s personal.,Iron Man,2008,8
2,TONY STARK,"Good God, you’re a woman. I, honestly, I could...",Iron Man,2008,31
3,TONY STARK,Please.,Iron Man,2008,1
4,TONY STARK,Excellent question. Yes and no. March and I ha...,Iron Man,2008,28


In [6]:
df.groupby(['movie', 'year']).head(1)[['movie', 'year']]

,movie,year
0,Iron Man,2008
433,Iron Man 2,2010
1104,Thor,2011
1364,Captain America: The First Avenger,2011
1556,The Avengers,2012
2265,Iron Man 3,2013
2797,Thor: The Dark World,2013
3032,Captain America: The Winter Soldier,2014
3437,Avengers: Age of Ultron,2015
4006,Ant-Man,2015


In [5]:
df.sample(20)[['character', 'line', 'movie', 'year']].head(20)

,character,line,movie,year
4526,PETER PARKER,Mm-hmm.,Spider-Man: Homecoming,2017
4283,TONY STARK,"Alright, Spider-Man. Get me out of this.",Captain America: Civil War,2016
3915,TONY STARK,The spire's Vibranium. If I get Thor to hit it...,Avengers: Age of Ultron,2015
4637,PETER PARKER,"This is a mistake. Hey, let’s just go home.",Spider-Man: Homecoming,2017
6157,BRUCE BANNER,We didn’t say we were here to see-,Avengers: Endgame,2019
5877,NICK FURY,"Well, she's dead.",Captain Marvel,2019
3542,THOR,That's the whole story?,Avengers: Age of Ultron,2015
2385,PEPPER POTTS,It would be incredible.,Iron Man 3,2013
3606,THOR,"Yes, well that's, uh, that's a very, very inte...",Avengers: Age of Ultron,2015
1569,NICK FURY,That can be harmful. Where’s agent Barton?,The Avengers,2012


#### Google Universal Sentence Encoder

In [10]:
import tensorflow as tf
import tensorflow_hub as hub

from sklearn.metrics.pairwise import linear_kernel

In [11]:
#Model load through URL path:

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)

#Create function for using model training
def embed(input):
    return model(input)

In [12]:
## Training
Model_USE= embed(df.line)

## Save model
exported = tf.train.Checkpoint(v=tf.Variable(Model_USE))
exported.f = tf.function(
    lambda  x: exported.v * x,
    input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])
tf.saved_model.save(exported,os.getcwd()+'\\pretrained_model\\' + 'TrainModel')

INFO:tensorflow:Assets written to: C:\Users\ramos\OneDrive\Documents\SyracuseUniversity\9th_Quarter\IST736\Project\pretrained_model\TrainModel\assets


INFO:tensorflow:Assets written to: C:\Users\ramos\OneDrive\Documents\SyracuseUniversity\9th_Quarter\IST736\Project\pretrained_model\TrainModel\assets


In [13]:
## Function for Document Search
def SearchDocument(query, topn=10):
    
    ## Create Query
    q =[query]
    
    # embed the query for calcluating the similarity
    Q_Train = embed(q)
    
    ## Load Model
    imported_m = tf.saved_model.load(os.getcwd()+'\\pretrained_model\\' + 'TrainModel')
    loadedmodel = imported_m.v.numpy()
    
    # Calculate the Similarity
    linear_similarities = linear_kernel(Q_Train, loadedmodel).flatten()
    # return linear_similarities

    #Sort top 10 index with similarity score
    Top_index_doc = linear_similarities.argsort()[:-(topn + 1):-1]
     
    # sort by similarity score
    linear_similarities.sort()
    a = df.loc[df.index.isin(Top_index_doc)].reset_index(drop=True)
    a['Score'] = linear_similarities[:-(topn+1):-1]
    
    return a.sort_values(['Score'], ascending=False)[['character', 'line', 'movie', 'year', 'Score']]

In [19]:
SearchDocument('character dies')

,character,line,movie,year,Score
0,JAMES RHODES,Vanko’s alive?,Iron Man 2,2010,0.414177
1,LOKI,Father is dead.,Thor,2011,0.372609
2,STEVE ROGERS,You lose someone?,Captain America: The Winter Soldier,2014,0.355951
3,STEVE ROGERS,Stark!,Avengers: Age of Ultron,2015,0.354970
4,STEVE ROGERS,Fatalities?,Avengers: Age of Ultron,2015,0.347357
5,STEVE ROGERS,Suit up.,Captain America: Civil War,2016,0.343662
6,PETER PARKER,Footage?,Spider-Man: Homecoming,2017,0.339325
7,THOR,Both dead.,Avengers: Infinity War,2018,0.334100
8,THOR,Killed by a dark elf.,Avengers: Infinity War,2018,0.326829
9,THOR,Noobmaster again?,Avengers: Endgame,2019,0.326669


In [15]:
SearchDocument('thanos')

,character,line,movie,year,Score
0,BRUCE BANNER,Thanos is coming. He's coming...,Avengers: Infinity War,2018,0.827228
1,THOR,You seem to know a great deal about Thanos.,Avengers: Infinity War,2018,0.813733
2,THOR,The Thanos killing kind.,Avengers: Infinity War,2018,0.735027
3,BRUCE BANNER,"There's an Ant-Man and a Spider-Man? Okay, loo...",Avengers: Infinity War,2018,0.734338
4,PETER PARKER,"So, you're not with Thanos?",Avengers: Infinity War,2018,0.726994
5,THOR,So is facing Thanos without that ax.,Avengers: Infinity War,2018,0.690775
6,THOR,BRING ME THANOS!,Avengers: Infinity War,2018,0.681241
7,JAMES RHODES,It’s been twenty-three days since Thanos came ...,Avengers: Endgame,2019,0.673123
8,BRUCE BANNER,Like Thanos.,Avengers: Endgame,2019,0.636227
9,BRUCE BANNER,Strange gave Thanos the Time Stone.,Avengers: Endgame,2019,0.629709


In [16]:
SearchDocument('hulk')

,character,line,movie,year,Score
0,TONY STARK,We have a Hulk.,The Avengers,2012,0.749868
1,NATASHA ROMANOFF,"Thor, report on the Hulk?",Avengers: Age of Ultron,2015,0.740821
2,BRUCE BANNER,"The world just saw the Hulk. The real Hulk, fo...",Avengers: Age of Ultron,2015,0.695066
3,STEVE ROGERS,Avengers...!,Avengers: Age of Ultron,2015,0.676475
4,PETER PARKER,The Avengers?,Spider-Man: Homecoming,2017,0.641309
5,THOR,"Hulk, I need you to do something for me.",Thor: Ragnarok,2017,0.604614
6,BRUCE BANNER,What are you saying? I’ve been Hulk for two ye...,Thor: Ragnarok,2017,0.602069
7,THOR,"So do I, but we have no choice. Surtur destroy...",Thor: Ragnarok,2017,0.580821
8,LOKI,"Well, for one thing, I'm not Asgardian. And fo...",Avengers: Infinity War,2018,0.574890
9,BRUCE BANNER,"Oh no, oh no you don't. This isn't going to be...",Avengers: Infinity War,2018,0.558523
